# Translate `dzn` to `smt2`

### Check Versions of Tools

In [1]:
import subprocess

output = subprocess.check_output("../optimathsat/bin/optimathsat -version", shell=True, universal_newlines=True)
output

'MathSAT5 version 1.6.4 (d39a648d6bb7) (Feb 25 2020 14:02:07, gmp 6.1.2, gcc 9.2.1, 64-bit)\n'

In [2]:
output = subprocess.check_output("../minizinc/bin/minizinc --version", shell=True, universal_newlines=True)
output

'MiniZinc to FlatZinc converter, version 2.4.3, build 122680178\nCopyright (C) 2014-2020 Monash University, NICTA, Data61\n'

In [3]:
output = subprocess.check_output("../z3/build/z3 --version", shell=True, universal_newlines=True)
output

'Z3 version 4.8.8 - 64 bit\n'

First generate the FlatZinc files using the MiniZinc tool. Make sure that a `smt2` folder is located inside `./minizinc/share/minizinc/`. Else, to enable OptiMathSAT's support for global constraints download the [smt2.tar.gz](http://optimathsat.disi.unitn.it/data/smt2.tar.gz) package and unpack it there using

```zsh
tar xf smt2.tar.gz $MINIZINC_PATH/share/minizinc/
```

If next output shows a list of `.mzn` files, then this dependency is satified.

In [3]:
output = subprocess.check_output("ls -la ../minizinc/share/minizinc/smt2/", shell=True, universal_newlines=True)
print(output)

total 292
drwxr-xr-x  2 kw kw 4096 Jan 15  2018 .
drwxr-xr-x 15 kw kw 4096 Apr 14 16:52 ..
-rw-r--r--  1 kw kw  328 Nov 13  2017 alldifferent_except_0.mzn
-rw-r--r--  1 kw kw  382 Nov 13  2017 all_different_int.mzn
-rw-r--r--  1 kw kw  396 Nov 13  2017 all_different_set.mzn
-rw-r--r--  1 kw kw  270 Nov 13  2017 all_disjoint.mzn
-rw-r--r--  1 kw kw  150 Nov 14  2017 all_equal_int.mzn
-rw-r--r--  1 kw kw  164 Nov 13  2017 all_equal_set.mzn
-rw-r--r--  1 kw kw  351 Nov 13  2017 among.mzn
-rw-r--r--  1 kw kw  305 Nov  8  2017 arg_max_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_max_int.mzn
-rw-r--r--  1 kw kw  306 Nov  8  2017 arg_min_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_min_int.mzn
-rw-r--r--  1 kw kw  480 Nov 13  2017 at_least_int.mzn
-rw-r--r--  1 kw kw  506 Nov 14  2017 at_least_set.mzn
-rw-r--r--  1 kw kw  340 Nov 13  2017 at_most1.mzn
-rw-r--r--  1 kw kw  474 Nov 13  2017 at_most_int.mzn
-rw-r--r--  1 kw kw  502 Nov 13  2017 at_most_set.mzn
-rw-r--r--  1 kw kw 1

## Transform `dzn` to `fzn` Using Model `mzn`

Then transform the desired `.dzn` file to `.fzn` using the `Mz.mzn` MiniZinc model.

In [4]:
import os

dzn_files = []
dzn_path = 'dzn-files/'

for filename in os.listdir(dzn_path):
    if filename.endswith(".dzn"):
        dzn_files.append(filename)
len(dzn_files)

278

In [16]:
fzn_path = 'fzn-files/z3/'
minizinc_base_cmd = '../minizinc/bin/minizinc \
    --compile --solver org.minizinc.mzn-fzn \
    --globals-dir smt2 mzn-model/Mz.mzn '
translate_count = 0
for dzn in dzn_files:
    translate_count += 1
    minizinc_transform_cmd = minizinc_base_cmd + dzn_path + dzn \
        + ' --output-to-file ' + fzn_path + dzn.replace('.', '-') + '.fzn'
    print(f'''({translate_count}/{len(dzn_files)}) Translating {dzn_path + dzn} to {fzn_path + dzn.replace('.', '-')}.fzn''')
    subprocess.check_output(minizinc_transform_cmd, shell=True, 
                                     universal_newlines=True)

(1/278) Translating dzn-files/R185.dzn to fzn-files/z3/R185-dzn.fzn
(2/278) Translating dzn-files/R191.dzn to fzn-files/z3/R191-dzn.fzn
(3/278) Translating dzn-files/A073.dzn to fzn-files/z3/A073-dzn.fzn
(4/278) Translating dzn-files/R137.dzn to fzn-files/z3/R137-dzn.fzn
(5/278) Translating dzn-files/R111.dzn to fzn-files/z3/R111-dzn.fzn
(6/278) Translating dzn-files/R168.dzn to fzn-files/z3/R168-dzn.fzn
(7/278) Translating dzn-files/R204.dzn to fzn-files/z3/R204-dzn.fzn
(8/278) Translating dzn-files/A023.dzn to fzn-files/z3/A023-dzn.fzn
(9/278) Translating dzn-files/A011.dzn to fzn-files/z3/A011-dzn.fzn
(10/278) Translating dzn-files/R039.dzn to fzn-files/z3/R039-dzn.fzn
(11/278) Translating dzn-files/A066.dzn to fzn-files/z3/A066-dzn.fzn
(12/278) Translating dzn-files/R026.dzn to fzn-files/z3/R026-dzn.fzn
(13/278) Translating dzn-files/R143.dzn to fzn-files/z3/R143-dzn.fzn
(14/278) Translating dzn-files/R112.dzn to fzn-files/z3/R112-dzn.fzn
(15/278) Translating dzn-files/R113.dzn to 

(121/278) Translating dzn-files/R149.dzn to fzn-files/z3/R149-dzn.fzn
(122/278) Translating dzn-files/A030.dzn to fzn-files/z3/A030-dzn.fzn
(123/278) Translating dzn-files/R079.dzn to fzn-files/z3/R079-dzn.fzn
(124/278) Translating dzn-files/R004.dzn to fzn-files/z3/R004-dzn.fzn
(125/278) Translating dzn-files/A037.dzn to fzn-files/z3/A037-dzn.fzn
(126/278) Translating dzn-files/A057.dzn to fzn-files/z3/A057-dzn.fzn
(127/278) Translating dzn-files/R129.dzn to fzn-files/z3/R129-dzn.fzn
(128/278) Translating dzn-files/R156.dzn to fzn-files/z3/R156-dzn.fzn
(129/278) Translating dzn-files/R125.dzn to fzn-files/z3/R125-dzn.fzn
(130/278) Translating dzn-files/R163.dzn to fzn-files/z3/R163-dzn.fzn
(131/278) Translating dzn-files/R040.dzn to fzn-files/z3/R040-dzn.fzn
(132/278) Translating dzn-files/R074.dzn to fzn-files/z3/R074-dzn.fzn
(133/278) Translating dzn-files/R203.dzn to fzn-files/z3/R203-dzn.fzn
(134/278) Translating dzn-files/R102.dzn to fzn-files/z3/R102-dzn.fzn
(135/278) Translatin

(239/278) Translating dzn-files/R116.dzn to fzn-files/z3/R116-dzn.fzn
(240/278) Translating dzn-files/R024.dzn to fzn-files/z3/R024-dzn.fzn
(241/278) Translating dzn-files/A045.dzn to fzn-files/z3/A045-dzn.fzn
(242/278) Translating dzn-files/R006.dzn to fzn-files/z3/R006-dzn.fzn
(243/278) Translating dzn-files/R169.dzn to fzn-files/z3/R169-dzn.fzn
(244/278) Translating dzn-files/A039.dzn to fzn-files/z3/A039-dzn.fzn
(245/278) Translating dzn-files/R199.dzn to fzn-files/z3/R199-dzn.fzn
(246/278) Translating dzn-files/R177.dzn to fzn-files/z3/R177-dzn.fzn
(247/278) Translating dzn-files/A064.dzn to fzn-files/z3/A064-dzn.fzn
(248/278) Translating dzn-files/R136.dzn to fzn-files/z3/R136-dzn.fzn
(249/278) Translating dzn-files/R132.dzn to fzn-files/z3/R132-dzn.fzn
(250/278) Translating dzn-files/R012.dzn to fzn-files/z3/R012-dzn.fzn
(251/278) Translating dzn-files/R109.dzn to fzn-files/z3/R109-dzn.fzn
(252/278) Translating dzn-files/R150.dzn to fzn-files/z3/R150-dzn.fzn
(253/278) Translatin

## Translate `fzn` to `smt2`

The generated `.fzn` file can be used to generate a `.smt2` file using the `fzn2smt2.py` script from this [project](https://github.com/PatrickTrentin88/fzn2omt).

In [36]:
import os

fzn_files = []
fzn_path = 'fzn-files/z3/'

for filename in os.listdir(fzn_path):
    if filename.endswith(".fzn"):
        fzn_files.append(filename)
len(fzn_files)

278

In [37]:
smt2_path = 'smt2-files/z3/'
fzn2smt2_base_cmd = '../fzn2omt/fzn2smt2.py --solver optimathsat --smt2 '
translate_count = 0
for fzn in fzn_files:
    translate_count += 1
    fzn2smt2_transform_cmd = fzn2smt2_base_cmd + smt2_path \
    + fzn.replace('.', '-') + '.smt2 ' + fzn_path + fzn
    print(f'''({translate_count}/{len(fzn_files)}) Translating {fzn_path + fzn} to {smt2_path + fzn.replace('.', '-')}.smt2''')
    my_env = os.environ.copy()
    my_env['PATH'] = "/home/kw/optimathsat/bin/:" + my_env['PATH']
    subprocess.check_output(fzn2smt2_transform_cmd, shell=True, env=my_env,
                                     universal_newlines=True)

(1/278) Translating fzn-files/z3/R074-dzn.fzn to smt2-files/z3/R074-dzn-fzn.smt2
(2/278) Translating fzn-files/z3/A003-dzn.fzn to smt2-files/z3/A003-dzn-fzn.smt2
(3/278) Translating fzn-files/z3/R194-dzn.fzn to smt2-files/z3/R194-dzn-fzn.smt2
(4/278) Translating fzn-files/z3/A068-dzn.fzn to smt2-files/z3/A068-dzn-fzn.smt2
(5/278) Translating fzn-files/z3/R192-dzn.fzn to smt2-files/z3/R192-dzn-fzn.smt2
(6/278) Translating fzn-files/z3/R033-dzn.fzn to smt2-files/z3/R033-dzn-fzn.smt2
(7/278) Translating fzn-files/z3/R177-dzn.fzn to smt2-files/z3/R177-dzn-fzn.smt2
(8/278) Translating fzn-files/z3/A071-dzn.fzn to smt2-files/z3/A071-dzn-fzn.smt2
(9/278) Translating fzn-files/z3/A005-dzn.fzn to smt2-files/z3/A005-dzn-fzn.smt2
(10/278) Translating fzn-files/z3/A008-dzn.fzn to smt2-files/z3/A008-dzn-fzn.smt2
(11/278) Translating fzn-files/z3/A019-dzn.fzn to smt2-files/z3/A019-dzn-fzn.smt2
(12/278) Translating fzn-files/z3/R091-dzn.fzn to smt2-files/z3/R091-dzn-fzn.smt2
(13/278) Translating fzn-

KeyboardInterrupt: 

### Adjust `smt2` Files According to 7.5 in Paper

- Remove `OptiMathSAT` specific settings
- Add lower and upper bounds for the decision variable `pfc`

In [34]:
import os
import re

smt2_path = 'smt2-files/z3'

for file in os.listdir(smt2_path):
    if file.endswith(".smt2"):
        with open(smt2_path+'/'+file, 'r+') as myfile:
            data = "".join(line for line in myfile if not line.startswith('(set'))

        filename = os.path.splitext(file)[0]
        print(filename)

        newFile = open(os.path.join(smt2_path, filename +'_v3.smt2'),"w+")
        newFile.write(data)
        newFile.close()

        openFile = open(os.path.join(smt2_path, filename +'_v3.smt2'))
        data = openFile.readlines()
        additionalLines = data[-5:]
        data = data[:-5]
        openFile.close()

        newFile = open(os.path.join(smt2_path, filename +'_v3.smt2'),"w+")
        newFile.writelines([item for item in data])
        newFile.close()

        with open(os.path.join(smt2_path, filename +'_v3.smt2'),"r") as myfile:
            data = "".join(line for line in myfile if not line.startswith('(set'))
        newFile = open(os.path.join(smt2_path, filename +'_v3.smt2'),"w+")
        newFile.write(data)
        matches = re.findall(r'\(define-fun .\d\d \(\) Int (\d+)\)', data)
        try:
            nbChambers = int(matches[0])
            print(nbChambers)
            k = 1
            for i in range(1,nbChambers+1):
                lb = f'''(define-fun lbound{str(i)} () Bool (> X_INTRODUCED_{str(i-1)}_ 0))\n'''
                ub = f'''(define-fun ubound{str(i)} () Bool (<= X_INTRODUCED_{str(i-1)}_ {str(nbChambers)}))\n'''
                assertLb = f'''(assert lbound{str(i)})\n'''
                assertUb = f'''(assert ubound{str(i)})\n'''

                newFile.write(lb)
                newFile.write(ub)
                newFile.write(assertLb)
                newFile.write(assertUb)
        except:
            print(f'''Check {filename} for completeness - data missing?''')
        
        newFile.writelines([item for item in additionalLines])
        newFile.close()

A062-dzn-fzn
160
A040-dzn-fzn
40
R077-dzn-fzn
48
A010-dzn-fzn
12
A024-dzn-fzn
86
R205-dzn-fzn
36
R074-dzn-fzn
20
R062-dzn-fzn
44
A003-dzn-fzn
24
A035-dzn-fzn
78
R156-dzn-fzn
78
A047-dzn-fzn
148
A030-dzn-fzn
10
R172-dzn-fzn
22
R130-dzn-fzn
64
R054-dzn-fzn
45
R143-dzn-fzn
20
R003-dzn-fzn
36
R030-dzn-fzn
26
R022-dzn-fzn
26
R041-dzn-fzn
10
A002-dzn-fzn
24
R025-dzn-fzn
32
R135-dzn-fzn
24
R142-dzn-fzn
20
R087-dzn-fzn
60
R132-dzn-fzn
56
R059-dzn-fzn
36
A071-dzn-fzn
192
R146-dzn-fzn
24
R082-dzn-fzn
12
R099-dzn-fzn
34
R131-dzn-fzn
64
A066-dzn-fzn
170
R110-dzn-fzn
18
R115-dzn-fzn
30
R171-dzn-fzn
65
R120-dzn-fzn
32
R175-dzn-fzn
84
R111-dzn-fzn
24
A048-dzn-fzn
68
R186-dzn-fzn
69
R137-dzn-fzn
56
R145-dzn-fzn
24
R148-dzn-fzn
16
A037-dzn-fzn
86
R184-dzn-fzn
70
R068-dzn-fzn
54
R091-dzn-fzn
58
R191-dzn-fzn
69
A059-dzn-fzn
70
R196-dzn-fzn
110
A026-dzn-fzn
12
A052-dzn-fzn
70
R123-dzn-fzn
20
R129-dzn-fzn
44
A061-dzn-fzn
130
R165-dzn-fzn
3
R128-dzn-fzn
40
R122-dzn-fzn
20
R090-dzn-fzn
58
R083-dzn-fzn
24
A01

## Test Generated `smt2` Files Using `z3`

This shoud generate the `smt2` file without any error. If this was the case then the `z3` prover can be called on a file by running


```zsh
z3 output/A001-dzn-smt2-fzn.smt2 
```

yielding something similar to

```zsh
z3 output/A001-dzn-smt2-fzn.smt2                                                       
sat
(objectives
 (obj 41881)
)
(model 
  (define-fun X_INTRODUCED_981_ () Bool
    false)
  (define-fun X_INTRODUCED_348_ () Bool
    false)
   
   .....
```

In [ ]:
result = subprocess.check_output('../z3/build/z3 smt2-files/A012-dzn-fzn.smt2', 
                        shell=True, universal_newlines=True)
print(result)